Compare RSMD 

In [1]:
import torch
import requests
import os
import subprocess
import tempfile
import time
import sys
import random
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from ESM_pymol import compare_sequence_sets,filter_valid_sequences_np,batch_numbers_to_letters
from utils import read_fasta_alignment, remove_duplicate_sequences
import numpy as np


cwd=r'C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main'


file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
true_sequences=seq_data_test_filtered.T
filename="gill_gen_seqs_w_init_seq_Ns30000_r0.3_after_mod"
output_file = cwd + r"\CODE\AttentionDCA_python\src\PLM\gill_generated_sequences\\" +filename +".npy"
generated_sequences = np.load(output_file)
filename="gen_seqs_w_init_seq_Ns30000_r0.1_b1"
output_file_plm=cwd + r"\CODE\AttentionDCA_python\src\PLM\generated_sequences\\" +filename +".npy"
generated_sequences_plm=np.load(output_file_plm)
filename="mc_generated_sequences_randinit_100000"
output_file_plm=cwd + r"\CODE\AttentionDCA_python\src\PLM\mc_generated_sequences\\" +filename +".npy"
generated_sequences_mc=np.load(output_file_plm)

generated_sequences=filter_valid_sequences_np(batch_numbers_to_letters(generated_sequences[1000:]))#with gillespie
generated_sequences_plm=filter_valid_sequences_np(batch_numbers_to_letters(generated_sequences_plm[1000:]))
generated_sequences_mc=filter_valid_sequences_np(batch_numbers_to_letters(generated_sequences_mc[1000:]))
true_sequences=filter_valid_sequences_np(batch_numbers_to_letters(true_sequences))









Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.

✅ Valid sequences kept: 4706 / 29000

✅ Valid sequences kept: 13945 / 29000

✅ Valid sequences kept: 25912 / 99000

✅ Valid sequences kept: 4903 / 14502


In [2]:
# filename="rmsd_matrix_gill_true.npy"
# # rmsd_results = compare_sequence_sets(true_sequences[::100], generated_sequences[::100],"gen","true")
# # np.save(filename, rmsd_results)
# rmsd_results_gill_true = np.load(filename,allow_pickle=True)

# filename="rmsd_matrix_true_true.npy"
# # rmsd_results = compare_sequence_sets(true_sequences[50::100], true_sequences[::100],"true2","true")
# # np.save(filename, rmsd_results)
# #rmsd_results_true_true = np.load(filename)

# filename="rmsd_matrix_gill_gill.npy"
# rmsd_results = compare_sequence_sets(generated_sequences[125::250], generated_sequences[::250],"gill2","gen")
# np.save(filename, rmsd_results)
#rmsd_results_gill_gill = np.load(filename)

# filename="rmsd_matrix_true_plm.npy"
# rmsd_results = compare_sequence_sets(true_sequences[::200], generated_sequences_plm[::600],"true","plm1")
# np.save(filename, rmsd_results)
# #rmsd_results_plm_true = np.load(filename)

# filename="rmsd_matrix_plm_plm.npy"
# rmsd_results = compare_sequence_sets(generated_sequences_plm[350::700], generated_sequences_plm[::700],"plm2","plm1")
# np.save(filename, rmsd_results)
#rmsd_results_plm_true = np.load(filename)

filename="rmsd_matrix_mc_mc.npy"
rmsd_results = compare_sequence_sets(generated_sequences_mc[10750::750], generated_sequences_mc[10000::750],"mc1","mc2")
np.save(filename, rmsd_results)
#rmsd_results_plm_true = np.load(filename)

filename="rmsd_matrix_mc_true.npy"
rmsd_results = compare_sequence_sets(generated_sequences_mc[10750::750], true_sequences[::300],"mc2","true1")
np.save(filename, rmsd_results)
#rmsd_results_plm_true = np.load(filename)

21
22
(21, 22)
⚠️ Failed to parse RMSD line: PyMOL>print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line:     3:print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line: PyMOL>print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line:     3:print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line: PyMOL>print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line:     3:print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line: PyMOL>print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line:     3:print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line: PyMOL>print("CUSTOM_RMSD:", r[0]) — could not convert string to float: '",'
⚠️ Failed to parse RMSD line:  

In [17]:
def dict_to_matrix(rmsd_dict):
    # Find the size of the matrix
    max_i = max(key[0] for key in rmsd_dict.keys())
    max_j = max(key[1] for key in rmsd_dict.keys())
    size = max(max_i, max_j) + 1  # Because indexing starts at 0

    # Initialize matrix with placeholder
    placeholder = 100.0
    matrix = np.full((size, size), placeholder)

    # Fill in known values
    for (i, j), value in rmsd_dict.items():
        matrix[i, j] = value

    # Compute mean of actual values (not placeholder)
    valid_values = matrix[matrix != placeholder]
    if valid_values.size > 0:
        mean_value = np.mean(valid_values)
    else:
        mean_value = 0.0  # Fallback if no valid data

    # Replace placeholder with mean
    matrix[matrix == placeholder] = mean_value

    return matrix

filename="rmsd_matrix_gill_gill.npy"
rmsd_results_gill_gill = np.load(filename,allow_pickle=True)
#rmsd_matrix_gill_gill = dict_to_matrix(rmsd_results_gill_gill)
print(np.mean(rmsd_results_gill_gill),"+-",np.std(rmsd_results_gill_gill))

filename="rmsd_matrix_gill_true.npy"
rmsd_results_gill_true = np.load(filename,allow_pickle=True).item()
rmsd_matrix_gill_true = dict_to_matrix(rmsd_results_gill_true)
print(np.mean(rmsd_matrix_gill_true),"+-",np.std(rmsd_matrix_gill_true))
filename="rmsd_matrix_true_true.npy"
rmsd_results_true_true = np.load(filename,allow_pickle=True).item()
rmsd_matrix_true_true = dict_to_matrix(rmsd_results_true_true)
print(np.mean(rmsd_matrix_true_true),"+-",np.std(rmsd_matrix_true_true))
filename="rmsd_matrix_true_plm.npy"
rmsd_results_true_plm = np.load(filename,allow_pickle=True).item()
rmsd_matrix_true_plm = dict_to_matrix(rmsd_results_true_plm)
print(np.mean(rmsd_matrix_true_plm),"+-",np.std(rmsd_matrix_true_plm))
filename="rmsd_matrix_plm_plm.npy"
rmsd_results_plm_plm = np.load(filename)
print(np.mean(rmsd_results_plm_plm),"+-",np.std(rmsd_results_plm_plm))
filename="rmsd_matrix_mc_mc.npy"
rmsd_results_mc_mc = np.load(filename)
print(np.mean(rmsd_results_mc_mc),"+-",np.std(rmsd_results_mc_mc))
filename="rmsd_matrix_mc_true.npy"
rmsd_results_true_mc = np.load(filename,allow_pickle=True)
print(np.mean(rmsd_results_true_mc),"+-",np.std(rmsd_results_true_mc))

1.2502802948046918 +- 2.0278676811052736
0.8355153639705813 +- 1.143644681346732
0.46134840163649343 +- 0.16451458226477042
0.6684451298539837 +- 0.8866438520817658
1.4468586384868554 +- 2.295894507231624
0.4776576412317557 +- 0.1760039631468965
0.5052709099875778 +- 0.15477114972782083
